# CREATING FLAG PARAMETER

In [0]:
dbutils.widgets.text("Incremental_Flag", '0')

In [0]:
Incremental_flag = dbutils.widgets.get("Incremental_Flag")

In [0]:
from pyspark.sql.functions import *

# CREATING DIMENSION MODEL

Fetch Relative Columns

In [0]:
%sql
SELECT * FROM parquet.`abfss://silver@carprojectdatalakejess.dfs.core.windows.net/car_sales`

In [0]:
df_src = spark.sql("""
SELECT DISTINCT Model_ID, Model_Category FROM parquet.`abfss://silver@carprojectdatalakejess.dfs.core.windows.net/car_sales`""")
    


In [0]:
df_src.display()

dim_model_Sink - Initial and Incremental

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_model'):
    df_sink = spark.sql(
    '''
    SELECT  dim_model_key, Model_ID, Model_Category FROM cars_catalog.gold.dim_model
    '''
)
else:
      df_sink = spark.sql(
    '''
    SELECT 1 as dim_model_key, Model_ID, Model_Category FROM parquet.`abfss://silver@carprojectdatalakejess.dfs.core.windows.net/car_sales` WHERE 1=0
    '''
)


In [0]:
df_sink.display()

# Filtering new records and old records

In [0]:
df_filter = df_src.join(df_sink, df_src['Model_ID'] == df_sink['Model_ID'], 'left')\
  .select(df_src['Model_ID'], df_src['Model_Category'], df_sink['dim_model_key'])
df_filter.display()

df_filter_old

In [0]:
df_filter_old = df_filter.filter(col("dim_model_key").isNotNull())
df_filter_old.display()


In [0]:
df_filter_new = df_filter.filter(col("dim_model_key").isNull()).select(df_src['Model_ID'], df_src['Model_Category'])

In [0]:
df_filter_new.display()

# Create Surrogate Key


Fetch the max Surrogate Key from existing table

In [0]:
if (Incremental_flag == '0'):
  max_value = 1
else:
  max_value_df = spark.sql("SELECT MAX(dim_model_key) FROM cars_catalog.gold.dim_model")
  max_value = max_value_df.collect()[0][0] +1

Create Surrogate Key Coulmn and ADD the max surrogate Key

In [0]:
df_filter_new = df_filter_new.withColumn("dim_model_key", max_value + monotonically_increasing_id())
df_filter_new.display()


Creating Final DF- df_filter_old + df_filter_new

In [0]:
df_final = df_filter_new.union(df_filter_old)

In [0]:
df_final.display()

SCD TYPE 1 (UPSERT)

In [0]:
from delta.tables import DeltaTable

In [0]:
#Incremental RUN
if spark.catalog.tableExists("cars_catalog.gold.dim_model"):
    delta_tbl = DeltaTable.forPath(spark, "abfss://gold@carprojectdatalakejess.dfs.core.windows.net/dim_model")
    delta_tbl.alias("trg").merge(df_final.alias("src"), "trg.dim_model_key = src.dim_model_key")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
  #Inital RUN
else:
    df_final.write.format("delta").mode("overwrite").option("path", "abfss://gold@carprojectdatalakejess.dfs.core.windows.net/dim_model").saveAsTable("cars_catalog.gold.dim_model")

In [0]:
%sql
SELECT * FROM cars_catalog.gold.dim_model